In [1]:
import lancedb
db_path = "C:\\Lee\\work\\contract\\db\\tmp\\contact_vectors.lancedb"
# --- 1. 连接/创建 LanceDB 数据库 ---
db = lancedb.connect(db_path)
print(f"LanceDB connected at: {db.uri}")

LanceDB connected at: C:\Lee\work\contract\db\tmp\contact_vectors.lancedb


In [2]:
# --- 2. 打开表 ---
table_name = "contact_table" # 你之前创建的表名
try:
    table = db.open_table(table_name)
    print(f"Successfully opened table: '{table.name}'")
    print(f"Table schema: {table.schema}")
    print(f"Number of rows in table: {len(table)}")
except Exception as e:
    print(f"Error opening table '{table_name}': {e}")
    print(f"Available tables: {db.table_names()}")
    exit()

Successfully opened table: 'contact_table'
Table schema: vector: fixed_size_list<item: float>[1024]
  child 0, item: float
id: string
payload: string
Number of rows in table: 50


In [3]:
import json

def read_and_decode_table(db, table_name):
    table = db.open_table(table_name)
    df = table.to_pandas()
    
    if 'payload' not in df.columns:
        raise ValueError("DataFrame中缺少'payload'列")
    
    # 解析payload字符串为字典
    df['parsed_payload'] = df['payload'].apply(
        lambda x: json.loads(x) if isinstance(x, str) else x
    )
    
    # 直接提取已解码的字段值
    df['name_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('name') if isinstance(x, dict) else None
    )
    
    df['content_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('content') if isinstance(x, dict) else None
    )
    
    # 可选：删除临时解析列
    df = df.drop(columns=['parsed_payload'])
    
    return df

In [4]:
data = read_and_decode_table(db, table_name)

In [8]:
data.iloc[0]['content_cn']

'买方合同编号：TGYR-2023-01卖方合同编号：订 货 合 同项目名称：山东泰山钢铁集团有限公司265㎡烧环冷机余热系统改造工程EPC总承包余热回收项目子项名称：环冷机余热锅炉买方：湖南中冶长天节能环保技术有限公司卖方：江苏东九重工股份有限公司合同签订时期：2023年 8 月合同签订地点：湖南·长沙合 同 双 方<table><tr><td>买\u3000\u3000方（印章）</td><td>卖\u3000\u3000方（印章）</td></tr><tr><td>名   称：湖南中冶长天节能环保技术有限公司</td><td>名  称：江苏东九重工股份有限公司</td></tr><tr><td>地    址：长沙市岳麓区节庆路7号</td><td>地址：盐城高新区华锐南路9号</td></tr><tr><td>联 系 人：戴亮</td><td>联 系 人：刘忠</td></tr><tr><td>电    话：18773083301</td><td>电    话：0515-88597534</td></tr><tr><td>传    真：</td><td>传    真：0515-88597987</td></tr><tr><td>电子邮件：1203852335@qq.com</td><td>电子邮件：Liuz118@163.com</td></tr><tr><td>开户银行：浦发银行长沙东塘支行</td><td>开户银行： 盐城市中行亭湖支行</td></tr><tr><td>帐    号：6605 0155 2000 0246 8</td><td>帐    号：5456 5820 7770</td></tr><tr><td>纳税人登记号：9143 0100 5849 3952 00</td><td>纳税人登记号：91320900140334109F</td></tr><tr><td>单位名称：湖南中冶长天节能环保技术有限公司</td><td>单位名称： 江苏东九重工股份有限公司</td></tr><tr><td>委托代理人：</td><td>委托代理人： </td></tr><tr><td>签字日期：</td><td>签字日期：</td></tr></table>3.1  最终供货一览表<table><thead><tr><th>序号</th><th cols